In [813]:
import pandas as pd
import numpy as np
import os
from datetime import date,timedelta

In [785]:
folder = r'C:\Users\jcallomamanib\Documents\GitProjects\maintenanceApp\02. Data2HR\data'
asset_data = 'AssetData.csv'
asset_system = 'AssetStructureA&S.csv'
system_component = 'AssetStructureS&C.csv'
component_dutie = 'DutieStructureC&D.csv'
dutie_data = 'DutieStructureD&D.csv'
dutie_resource = 'DutieStructureD&R.csv'
wbs = 'WBS.csv'

ruteA_D = os.path.join(folder,asset_data)
ruteA_S = os.path.join(folder,asset_system)
ruteS_C = os.path.join(folder,system_component)
ruteC_D = os.path.join(folder,component_dutie)
ruteD_D = os.path.join(folder,dutie_data)
ruteD_R = os.path.join(folder,dutie_resource)
rute_wbs = os.path.join(folder,wbs)


In [786]:
dataA_D = pd.read_csv(ruteA_D, dtype = {'Cod sap':str})
dataA_S = pd.read_csv(ruteA_S, dtype = {'System':str})
dataS_C = pd.read_csv(ruteS_C)
dataC_D = pd.read_csv(ruteC_D)
dataD_D = pd.read_csv(ruteD_D, dtype = {'Specialist':str,'Amount Craft':int,'Time':float})
dataD_R = pd.read_csv(ruteD_R, dtype = {'Cod. Material':str})
data_wbs = pd.read_csv(rute_wbs)

In [787]:
## DATA OF DUTIES AND TASK INFORMATION
# All the conditional activities are remove
dataD_D_1task = dataD_D.loc[dataD_D.loc[:,'Maintenance type'] != 'COND',:].reset_index(drop = True)
dataD_D_2task = dataD_D.loc[dataD_D.loc[:,'Maintenance type'] == 'COND',:].reset_index(drop = True)
# Adding Date of last Task done
dataD_D_1task.loc[:, 'Date task done'] = date(2021,4,1)
# All the non Mechanical or Predective activities are remove
dataD_D_1task.loc[:,'Man hour'] = dataD_D_1task.loc[:,'Time']*dataD_D_1task.loc[:,'Amount Craft']
# Adding Estrategy to the activities
dataD_D_1task.loc[:,'Strategy'] = dataD_D_1task.loc[:,'Frequency'].astype(int).astype(str)+dataD_D_1task.loc[:,'Unidad Periodo']

dataD_D_1T_MP = dataD_D_1task.loc[(dataD_D_1task.loc[:,'Specialist']=='TEC-MEC') | (dataD_D_1task.loc[:,'Specialist']=='TEC-PRD'),:].reset_index(drop = True)
dataD_D_1T_M = dataD_D_1task.loc[dataD_D_1task.loc[:,'Specialist']=='TEC-MEC',:].reset_index(drop = True)
dataD_D_1T_P = dataD_D_1task.loc[dataD_D_1task.loc[:,'Specialist']=='TEC-PRD',:].reset_index(drop = True)
# load the data to work on
dataD_D_1T_M.to_csv('dataD_D_1T_M.csv',encoding='utf-8-sig')
dataD_D_1T_M.to_excel('dataD_D_1T_M.xlsx',encoding='utf-8-sig')
dataD_D_1T_P.to_csv('dataD_D_1T_P.csv',encoding='utf-8-sig')
dataD_D_1T_P.to_excel('dataD_D_1T_P.xlsx',encoding='utf-8-sig')

In [788]:
# DATA OF ASSET INFORMATION
# Removing Spanish information
asset_SP_EN = ['WBS_1_ES', 'WBS_2_ES', 'WBS_3_ES']
dataA_D_EN = dataA_D.drop(asset_SP_EN, axis=1)
# All Johan&Jorge clearence
dataA_D_JJ = dataA_D_EN.loc[dataA_D.loc[:,'Responsable']=='Johan&Jorge',:].sort_values(by='Asset tag').reset_index(drop = True)

In [789]:
%%time
# Merging all datas csv 
# Asset&Data(Johan&Jorge) -> Asset&System -> System&Component -> Component&Duties -> 
# Duties&Data(Mechanical/Predective/Mechanical&Predective)
dataA_D_JJ_S = pd.merge(dataA_D_JJ,dataA_S,how = 'left',on = 'Asset tag')
dataA_S_C_JJ = pd.merge(dataA_D_JJ_S, dataS_C, how='left', on = 'key_system')
dataA_S_C_D_JJ = pd.merge(dataA_S_C_JJ, dataC_D, how='left', on='key_component')
dataAllcompileMP = pd.merge(dataA_S_C_D_JJ, dataD_D_1T_MP, how = 'inner', on = 'key_dutie')
dataAllcompileP = pd.merge(dataA_S_C_D_JJ, dataD_D_1T_P, how = 'inner', on = 'key_dutie')
dataAllcompileM = pd.merge(dataA_S_C_D_JJ, dataD_D_1T_M, how = 'inner', on = 'key_dutie')

Wall time: 88.8 ms


In [790]:
# Saving all data compile Predective and Mechanical
dataAllcompileP.to_csv('data_all_prd.csv', encoding = 'utf-8-sig')
dataAllcompileM.to_csv('data_all_mec.csv', encoding = 'utf-8-sig')

In [791]:
dataAllcompileM.columns

Index(['Asset tag', 'Cod sap', 'WBS_1_EN', 'WBS_2_EN', 'WBS_3_EN',
       'Responsable', 'Hour day', 'key_system', 'System', 'key_component',
       'Component', 'key_dutie', 'Primary task',
       'Primary task detail activity', 'Maintenance type', 'Specialist',
       'Constraint', 'Time', 'Amount Craft', 'Frequency', 'Unidad Periodo',
       'Date task done', 'Man hour', 'Strategy'],
      dtype='object')

In [792]:
dataD_D_1T_M.columns

Index(['key_dutie', 'Primary task detail activity', 'Maintenance type',
       'Specialist', 'Constraint', 'Time', 'Amount Craft', 'Frequency',
       'Unidad Periodo', 'Date task done', 'Man hour', 'Strategy'],
      dtype='object')

In [848]:
columnfilter = ['key_system', 'Component','Primary task', 'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft','Frequency', 'Unidad Periodo','Strategy', 'Date task done']
projection = dataAllcompileM.loc[:,['Asset tag','Strategy','Frequency', 'Unidad Periodo','Date task done','Hour day']].groupby(['Asset tag','Strategy','Frequency', 'Unidad Periodo','Date task done','Hour day']).size().reset_index()
projection.drop(columns = {0},inplace =True)

In [849]:
[row,column] = projection.shape

In [851]:
projection.loc[:,'Date plan'] = projection.loc[:,'Date task done']

In [852]:
planDates = pd.DataFrame(columns = projection.columns)
planDates 

,Asset tag,Strategy,Frequency,Unidad Periodo,Date task done,Hour day,Date plan


In [853]:
projection.loc[0,'Date task done'] + timedelta(days = 0)

datetime.date(2021, 4, 1)

In [854]:
x = 0
for i in range(row):
    while x < 5:
        projection.loc[i,'Date plan'] = projection.loc[i,'Date plan'] + timedelta(hours = projection.loc[i,'Frequency'])
        planDates = planDates.append(projection.loc[i,:],ignore_index=True)
        x +=1


In [855]:
planDates

,Asset tag,Strategy,Frequency,Unidad Periodo,Date task done,Hour day,Date plan
0,2221-PU-201,1000H,1000.0,H,2021-04-01,24,2021-05-12
1,2221-PU-201,1000H,1000.0,H,2021-04-01,24,2021-06-22
2,2221-PU-201,1000H,1000.0,H,2021-04-01,24,2021-08-02
3,2221-PU-201,1000H,1000.0,H,2021-04-01,24,2021-09-12
4,2221-PU-201,1000H,1000.0,H,2021-04-01,24,2021-10-23


In [800]:
# DATA OF DATES AND PLANIFICATION
dataStrategy = dataAllcompileM.loc[:,['Frequency','Unidad Periodo']].groupby(['Frequency','Unidad Periodo']).size().reset_index()
dataStrategy.loc[:,0] = dataStrategy.loc[:,'Frequency'].astype(int).astype(str) + dataStrategy.loc[:,'Unidad Periodo']
dataStrategy.rename(columns = {0:'claveStrategy'},inplace =True)

In [801]:
dataStrategy.head()

,Frequency,Unidad Periodo,claveStrategy
0,1.0,A,1A
1,1.0,M,1M
2,1.0,S,1S
3,2.0,M,2M
4,3.0,M,3M


In [802]:
# Data wrangling
filtertype = dataAllcompileM.loc[:,'Maintenance type'] == 'INSP'
filterunidad = dataAllcompileM.loc[:,'Unidad Periodo'] == 'H'
filterhour = dataAllcompileM.loc[:,'Frequency'] == 1400
dataAllcompileM.loc[filtertype & filterunidad,'Frequency_eq'] = (dataAllcompileM.loc[filtertype & filterunidad,'Frequency']/(24*7)).round(0)

In [803]:
dataAllcompileM.loc[filtertype & filterunidad,['Frequency','Frequency_eq']].groupby(['Frequency','Frequency_eq']).size()

Frequency  Frequency_eq
165.0      1.0             242
330.0      2.0             101
660.0      4.0             711
1000.0     6.0              24
1400.0     8.0               2
1500.0     9.0               7
2000.0     12.0            168
3000.0     18.0              4
4000.0     24.0             89
6000.0     36.0              1
8000.0     48.0             79
dtype: int64

In [804]:
# Combine activities based on similar columns
columnfilter = ['key_system', 'Component','Primary task', 'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft','Frequency', 'Unidad Periodo']
# '2253-PU-212'
rowfilter = dataAllcompileM.loc[:,'Asset tag'] == '2221-PU-201'
# dataAllcompile.loc[rowfilter,columnfilter]
datacombine = dataAllcompileM.loc[rowfilter,columnfilter].groupby(['key_system','Component','Maintenance type','Constraint','Frequency']).agg({'Primary task': lambda x: "\n".join(x)}).reset_index()

In [805]:
datacombine

,key_system,Component,Maintenance type,Constraint,Frequency,Primary task
0,2221-PU-201|01,Acople de alta,INSP,MAR,165.0,Inspección acople de alta de bomba
1,2221-PU-201|01,Acople de baja,INSP,MAR,165.0,Inspección acople de baja de bomba
2,2221-PU-201|01,Reductor de velocidad,INSP,MAR,165.0,Inspección de reductor de bomba\nInspec nivel ...
3,2221-PU-201|01,Reductor de velocidad,PREV,DET,4000.0,Cambio aceite de reductor de bomba
4,2221-PU-201|01,Sistema de enfriamiento,INSP,MAR,660.0,Insp general sistema enfriamiento
5,2221-PU-201|01,Sistema de enfriamiento,PREV,DET,4000.0,Cambio filtros aceite sist enfriam
6,2221-PU-201|01,Sistema de enfriamiento,PREV,DET,8000.0,Cambio bomba aceite sist enfriam
7,2221-PU-201|02,Carcasa,INSP,DET,660.0,Inspección de carcasa de bomba
8,2221-PU-201|02,Carcasa,INSP,MAR,165.0,Inspección stuffing box de carcasa
9,2221-PU-201|02,Carcasa,PREV,DET,1000.0,Cambio spare parte húmeda de bomba\nCambio de ...
